In [1]:
import requests
from datetime import datetime, timedelta
import numpy as np

class Stock:
    url = "https://www.google.com/finance/getprices"
    interval = 60 * 10 #sec
    market = "TYO"
    period = "2d"
    column = "d,c"
    market_duration_minute = 360 #休憩を含む
    def __init__(self, code, label, name):
        self.code = code
        self.label = label
        self.name = name
        self.prices = []
        self.timestamps = []
        self.price_changes = []
        self.price_slope = []
        self.fetched = False
        
    

    def fetch_prices(self, reread=True):
        
        if not reread and self.fetched :
            return
        
        self.fetched = False
        
        params = {
            'q': self.code,
            'i': Stock.interval,
            'x': Stock.market,
            'p': Stock.period,
            'f': Stock.column
        }

        #http リクエスト
        r = requests.get(Stock.url, params=params)

        #print( r.text )
        
        #http レスポンスを行単位に切る
        lines = r.text.splitlines()
        columns = lines[4].split("=")[1].split(",")

        #ここからが株価
        prices = lines[8:]

        #レスポンスの１日目のタイムスタンプをdatetimeに
        base_time = 0
        self.price_changes = []
        self.price_slope = []
        self.prices = []
        self.timestamps = []
        for i in range( len(prices) ):
            cols = prices[ i ].split(",")
            if 'a' in cols[0]:
                if base_time != 0: break
                base_time = int( cols[0].lstrip('a') )
                d = datetime.fromtimestamp( base_time )
            else:
                ofst = int( cols[0] )
                if not self.within_a_day( ofst ):
                    break
                d = datetime.fromtimestamp( base_time + ofst * Stock.interval )
    
            if  self.just_begining_time(d) or self.just_end_time(d):
                #開始時間ピッタリのやつはおかしい。
                #終了時間ピッタリのやつはないのもあるので除外。
                pass
            else:
                self.prices.append( float(cols[1]) )
                self.timestamps.append(d.strftime('%Y-%m-%d %H:%M:%S'))
        
        for i in range( len(self.prices) - 1):
            change = self.prices[i+1] - self.prices[i]
            self.price_changes.append( change )

        chain = 0
        for i in range( len(self.price_changes) ):
            if self.price_changes[i] > 0:
                chain += 1
            else:
                chain = 0
            self.price_slope.append( float(chain) )
            
        self.fetched = True
        
    def _plot( self, xs ):
        x = np.arange( len(xs) )
        y = np.array( xs )
        return x, y
    
    def plot_price_xy( self ):
        return self._plot( self.prices )
    
    def plot_change_xy(self):
        s = 0
        lst = []
        for c in self.price_changes:
            s += c
            lst.append( s )
        return self._plot( lst )
    
    def plot_slope_xy(self):
        return self._plot( self.price_slope )
    
    def within_a_day( self, ofst ):
        return (ofst * Stock.interval) < (Stock.market_duration_minute * 60)
        
    def just_begining_time( self, d ):
        return (d.hour == 9 and d.minute == 0) or (d.hour == 12 and d.minute == 30 )
    
    def just_end_time( self, d ):
        return (d.hour == 15 and d.minute == 0 )
                

def get_stock( no ):
    s = Stock( no, "", "" )
    s.fetch_prices()
    return s

def print_stock():
    s0 = get_stock( 6758 )
    s1 = get_stock( 6762 )
    s2 = get_stock( 4503 )
    print( "{}:{} {}:{} {}:{}".format( len(s0.prices), len(s0.timestamps), len(s1.prices), len(s1.timestamps), len(s2.prices), len(s2.timestamps)))
    for i in range( len( s1.prices )):
        
        print( "-----" + str(i) + "----" )
        print( "{}  |  {}  |  {}".format( s0.timestamps[i], s1.timestamps[i], s2.timestamps[i]))
        print( "{}  |  {}  |  {}".format( s0.prices[i], s1.prices[i], s2.prices[i]))

if __name__ == '__main__':
    #print_stock()
    pass
    

In [13]:
prices = range( 100 )

start = len(prices)
ofst = -1
val = 300
try:
    next( p for p in prices[ start : : ofst ] if p == val )
except:
    print( "wrong")
    raise Exception( "wrong wrong")

wrong


Exception: wrong wrong

In [4]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import re

class Nikkei225:
    # アクセスするURL
    url = "https://indexes.nikkei.co.jp/nkave/index/component?idx=nk225"
    
    def __init__( self ):
        self.title = ""
        self.date = ""
        self.stocks = []
        self.soup = None
        
    def fetch_date( self ):
        divs = self.soup.find_all("div", class_="last-update" )
        assert len(divs)==1, "no last-update class found."
        self.date = divs[0].string
        
    def fetch_stock( self ):
        self.divs = self.soup.find_all("div", class_=re.compile("row component-list.*") )
        
        for tag in self.divs:
            # classの設定がされていない要素は、tag.get("class").pop(0)を行うことのできないでエラーとなるため、tryでエラーを回避する
            try:
                code_ = tag.find_all("div", class_="col-xs-3 col-sm-1_5")[0].string
                label_ = tag.find_all("div", class_="col-xs-9 col-sm-2_5")[0].string
                company_ = tag.find_all("div", class_="hidden-xs col-sm-8")[0].string
                self.stocks.append( Stock(code_, label_, company_) )
            except:
                # パス→何も処理を行わない
                pass
        
        
    def fetch( self ):
        # URLにアクセスする htmlが帰ってくる → <html><head><title>経済、株価、ビジネス、政治のニュース:日経電子版</title></head><body....
        html = urllib.request.urlopen(self.url)
        
        # htmlをBeautifulSoupで扱う
        self.soup = BeautifulSoup(html, "html.parser")

        # タイトル要素を取得する → <title>経済、株価、ビジネス、政治のニュース:日経電子版</title>
        title_tag = self.soup.title

        # 要素の文字列を取得する → 経済、株価、ビジネス、政治のニュース:日経電子版
        self.title = title_tag.string
              
        self.fetch_date()
        self.fetch_stock()
        
    def fetch_all_prices( self ):
        for s in self.stocks:
            s.fetch_prices()

def print_stocks():
    n = Nikkei225()
    n.fetch()
    print( n.title )
    print( n.date )
    for i, s in enumerate( n.stocks ):
        print( "{:3}: {}".format( i, s.label ) )
        
if __name__ == '__main__':
    #print_stocks()
    pass
    
                             
                             

In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import sys
import matplotlib.pyplot as plt

from ipywidgets import *

nikkei = Nikkei225()
nikkei.fetch()

#ラベル
menu_options = []
menu = None

for i, s in enumerate( nikkei.stocks ):
    menu_options.append( s.label + " (" + str(s.code) + ")" )
    
def minmax( l, margin = 4 ):
    min = l.min()
    max = l.max()
    return min-margin, max+margin

def plot_prices( selections ):
    y_min = sys.maxsize
    y_max = -sys.maxsize
    xs = []
    ys = []
    ls = []
    for i, s in enumerate(nikkei.stocks):
        if s != None and menu_options[i] in selections:
            #x,y = s.plot_price_xy()   
            #x,y = s.plot_change_xy()
            x,y = s.plot_slope_xy()
            xs.append( x )
            ys.append( y )
            ls.append( str(s.code) )
            min, max = minmax( y )
            if min < y_min: 
                y_min = min
            if max > y_max:
                y_max = max
    
    plt.figure(figsize=(20,10))
    for i in range( len(xs) ):
        plt.plot(xs[i],ys[i],label=ls[i])
    plt.legend( loc = "upper left")
    plt.grid( True )
    plt.ylim( y_min, y_max )
        
    plt.show()

def show_frames( selections ):
    df = None
    for lbl in selections:
        idx = menu_options.index( lbl )
        s = nikkei.stocks[ idx ]
        if df == None: 
            df = pd.DataFrame( s.prices )
        else:
            df.concat( pd.DataFrame( s.prices ) )
        
    display(df)
    
def on_value_change( selections ):
    #new
    for lbl in selections:
        idx = menu_options.index( lbl )
        nikkei.stocks[ idx ].fetch_prices( False )
    
    plot_prices( selections )
    show_frames( selections )

def create_menu():
    menu = SelectMultiple(options=menu_options)
    display( interactive(on_value_change, selections=menu) )

def fetch_all_prices():
    bar = FloatProgress()
    bar.value = 0
    bar.min = 0
    bar.max = 100
    bar.description = "Loading Nikkei 225..."
    display( bar )
    for i,s in enumerate(nikkei.stocks):
        s.fetch_prices()
        bar.value = 100 * i / float(len(nikkei.stocks))
    bar.close()
        
    
if __name__ == '__main__':
    fetch_all_prices()
    create_menu()

geo={'USA':['CHI','NYC'],'Russia':['MOW','LED']}

def print_city(city):
    print ( city )

def select_city(country):
    cityW.options = geo[country]

def test():
    scW = widgets.Dropdown(options=geo.keys())
    init = scW.value
    cityW = widgets.Dropdown(options=geo[init])
    j = widgets.interactive(print_city, city=cityW)
    i = widgets.interactive(select_city, country=scW)
    display(i)
    display(j)

        
    

FloatProgress(value=0.0, description='Loading Nikkei 225...')

interactive(children=(SelectMultiple(description='selections', options=('協和キリン (4151)', '武田 (4502)', 'アステラス (4…

In [8]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import sys
import matplotlib.pyplot as plt

from ipywidgets import *

nikkei = Nikkei225()
nikkei.fetch()
#ラベル
#menu_options = []
#menu = None

length = len( nikkei.stocks[0].price_slope )
chain_list = []
plot_types =["Change", "Combo"]
plot_type = 0
selection_labels = []
selection_index = 0

for i in range( length ):
    lst = []
    ts = nikkei.stocks[0].timestamps[ i ]
    for s in nikkei.stocks:
        if len(s.price_slope)==length and 5 <= s.price_slope[ i ] and s.price_changes[ i ] > 0 :
            lst.append( s )
    if len(lst ) > 2:
        chain_list.append( {"timestamp":ts, "stocks":lst} )

        
def plot_slope_prices( selections, start):
    y_min = sys.maxsize
    y_max = -sys.maxsize
    xs = []
    ys = []
    ls = []
    global plot_type
    for i, s in enumerate(nikkei.stocks):
        if s != None and menu_options[i] in selections:
            #x,y = s.plot_price_xy()  
            if plot_type == 0:
                x,y = s.plot_change_xy()
            else:
                x,y = s.plot_slope_xy()
            xs.append( x )
            ys.append( y )
            ls.append( str(s.code) )
            min, max = minmax( y )
            if min < y_min: 
                y_min = min
            if max > y_max:
                y_max = max
    diff = 5.0
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)
    rect = plt.Rectangle((start-diff,y_min),diff,y_max-y_min,fc="#c0c0c0")
    ax.add_patch(rect)
    for i in range( len(xs) ):
        plt.plot(xs[i],ys[i],label=ls[i])
    plt.legend( loc = "upper left")
    plt.grid( True )
    plt.ylim( y_min, y_max )
        
    plt.show()
    
def on_selection_change( selection ):
    #new
    for l in chain_list:
        if l["timestamp"] == selection:
            i = nikkei.stocks[0].timestamps.index(selection)
            sels = []
            lbls = ""
            for s in l["stocks"]:
                sels.append( s.label + " (" + str(s.code) + ")" )
                lbls += s.label +"--" + s.code + "(" + str(int(s.price_slope[i])) +"times) "
            print( lbls )
            plot_slope_prices( sels, i )
            global selection_labels
            global selection_index
            selection_labels = sels
            selection_index = i
            #show_frames( sels )
    
def on_change( selection ):
    global plot_type
    global selection_labels
    global selection_index
    plot_type = plot_types.index( selection )
#    plot_slope_prices( selection_labels, selection_index )
            
def create_sel_menu():
    opts = []
    for l in chain_list:
        opts.append(l["timestamp"])
    select_menu = Select(options=opts)
    display( interactive(on_selection_change, selection=select_menu) )
    toggle = ToggleButtons( options=plot_types)
    display( interactive(on_change, selection=toggle) )

def fetch_all_prices():
    bar = FloatProgress()
    bar.value = 0
    bar.min = 0
    bar.max = 100
    bar.description = "Loading Nikkei 225..."
    display( bar )
    for i,s in enumerate(nikkei.stocks):
        s.fetch_prices()
        bar.value = 100 * i / float(len(nikkei.stocks))
    bar.close()
    
fetch_all_prices()    
create_sel_menu()
    

FloatProgress(value=0.0, description='Loading Nikkei 225...')

interactive(children=(Select(description='selection', options=(), value=None), Output()), _dom_classes=('widge…

interactive(children=(ToggleButtons(description='selection', options=('Change', 'Combo'), value='Change'), Out…

In [5]:
#from urllib2 import *
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import pandas as pd


# アクセスするURL
url = "https://indexes.nikkei.co.jp/nkave/index/component?idx=nk225"

# URLにアクセスする htmlが帰ってくる → <html><head><title>経済、株価、ビジネス、政治のニュース:日経電子版</title></head><body....
html = urllib.request.urlopen(url)

# htmlをBeautifulSoupで扱う
soup = BeautifulSoup(html, "html.parser")

# タイトル要素を取得する → <title>経済、株価、ビジネス、政治のニュース:日経電子版</title>
title_tag = soup.title

# 要素の文字列を取得する → 経済、株価、ビジネス、政治のニュース:日経電子版
title = title_tag.string

# タイトル要素を出力
#print( title_tag )

# タイトルを文字列を出力
print( title )

divs = soup.find_all("div")

#help( divs[0] )

date = ""
stocks = []

for tag in divs:
    # classの設定がされていない要素は、tag.get("class").pop(0)を行うことのできないでエラーとなるため、tryでエラーを回避する
    try:
        # tagの中からclass="n"のnの文字列を摘出します。複数classが設定されている場合があるので
        # get関数では配列で帰ってくる。そのため配列の関数pop(0)により、配列の一番最初を摘出する
        # <span class="hoge" class="foo">  →   ["hoge","foo"]  →   hoge
        strings_ = tag.get("class")
        string_ = ""
        for s in strings_:
            string_ += s
            string_ += ' '#末尾に空白が入る。
            
        if string_ in "row component-list " or string_ in "row component-list list-even ":

            code_ = ""
            label_ = ""
            company_ = ""
            tags = tag.find_all("div")
            for child in tags:
                string_ = child.get("class").pop(0)
                
                #print( type( child.string ) )
                if string_ in "col-xs-3 col-sm-1_5": code_ = "{0}".format(child.string)
                if string_ in "col-xs-9 col-sm-2_5": label_ = "{0}".format(child.string)
                if string_ in "hidden-xs col-sm-8": company_ = "{0}".format(child.string)

            
            if len( code_ ) != 0:
                s = Stock( code_, label_, company_ )
                s.fetch_prices()
                #print( s.prices )
                l = [code_,label_]
                l = l + s.prices
                stocks.append( l )
                #stocks.append( [code_,label,company_] )
                #print( code_ + label_ + company_)
                #break

        elif string_ in "last-update ":
            date = tag.string
    except:
        # パス→何も処理を行わない
        pass
    
print( date )

#stocks

df = pd.DataFrame( stocks )
df.to_csv( "日経２２５ " + date + ".csv", index=False)
df


銘柄一覧 － 日経平均プロフィル
更新日付：2018.04.12


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,4151,協和キリン,2355.0,2359.0,2361.0,2361.0,2350.0,2348.0,2344.0,2343.0,...,2331.0,2336.0,2339.0,2336.0,2337.0,2340.0,2340.0,2341.0,2343.0,2342.0
1,4502,武田,5010.0,5028.0,5036.0,5034.0,5028.0,5030.0,5037.0,5043.0,...,5068.0,5068.0,5068.0,5061.0,5067.0,5068.0,5068.0,5068.0,5071.0,5075.0
2,4503,アステラス,1571.5,1583.0,1592.5,1593.0,1590.5,1589.0,1589.5,1589.0,...,1577.5,1578.0,1577.0,1577.5,1579.5,1582.5,1583.0,1583.5,1584.0,1584.0
3,4506,大日本住友,1957.0,1957.0,1961.0,1960.0,1954.0,1954.0,1957.0,1953.0,...,1935.0,1937.0,1937.0,1937.0,1937.0,1939.0,1939.0,1939.0,1939.0,1937.0
4,4507,塩野義,5471.0,5480.0,5522.0,5529.0,5517.0,5514.0,5505.0,5505.0,...,5506.0,5507.0,5507.0,5504.0,5504.0,5505.0,5503.0,5506.0,5518.0,5512.0
5,4519,中外薬,5620.0,5640.0,5680.0,5690.0,5670.0,5680.0,5660.0,5660.0,...,5570.0,5560.0,5560.0,5560.0,5570.0,5580.0,5580.0,5580.0,5610.0,5610.0
6,4523,エーザイ,6699.0,6666.0,6766.0,6784.0,6769.0,6748.0,6753.0,6736.0,...,6674.0,6687.0,6690.0,6673.0,6683.0,6684.0,6685.0,6697.0,6702.0,6699.0
7,4568,第一三共,3699.0,3700.0,3720.0,3718.0,3695.0,3696.0,3701.0,3704.0,...,3665.0,3665.0,3666.0,3661.0,3657.0,3664.0,3664.0,3672.0,3675.0,3676.0
8,4578,大塚ＨＤ,5313.0,5332.0,5373.0,5369.0,5355.0,5349.0,5347.0,5342.0,...,5316.0,5332.0,5332.0,5334.0,5335.0,5341.0,5343.0,5344.0,5349.0,5346.0
9,3105,日清紡ＨＤ,1448.0,1454.0,1443.0,1439.0,1440.0,1445.0,1450.0,1449.0,...,1443.0,1444.0,1443.0,1442.0,1442.0,1443.0,1442.0,1442.0,1442.0,1442.0


In [6]:
s = ""
s = "{0}".format( "test")
print( s )

test
